# Решение небольших тестовых примеров

Содержание:

**Одно ТС небольшой вместимости доставляет в 2 города.**

**В одном городе 2 заказа.**

**4 ТС и 6 городов**

```
Во всех примерах база - Moscow. Остальные города видны в таблице заказов.

Начало доставки = 0

Окончание доставки = 25
```

In [1]:
import sys
import pandas as pd

sys.path.append("vehicle_routing/")

from data_reader import DataReader
from data_processor import DataProcessor
from data_writer import DataWriter
from linprog_maker import LinProgMaker

# Одно ТС небольшой вместимости доставляет в 2 города

In [2]:
in_file = "tests/data/test01_v1c3dhuge.xlsx"
out_file = "no_file.xlsx"

In [3]:
reader = DataReader(in_file)
processor = DataProcessor(reader)
task = LinProgMaker(processor)
task.solve()
writer = DataWriter(task, out_file)

## Данные

#### Заказы - покупатели

In [4]:
reader.read_orders()

,City,Demand,Price_per_unit,Deliver_from,Deliver_to
0,Tambov,200,800,3,15
1,Voronezh,100,1100,17,24


#### ТС - машины

In [5]:
reader.read_vehicles()

,CarName,Capacity,Price_per_km
0,Nissan,60,9


#### Маршруты

In [6]:
reader.read_routes()

,From,To,Time,Distance
0,Moscow,Tambov,3.000000,400
1,Moscow,Voronezh,5.000000,450
2,Tambov,Moscow,3.000000,400
3,Tambov,Voronezh,2.111111,190
4,Voronezh,Tambov,2.111111,190
5,Voronezh,Moscow,5.000000,450


## Решение и комментарии

#### Таблица - Распределение маршрутов

In [7]:
writer.make_vehicle_routing()

,CarName,TripNumber,City_From,City_To,Delivered,Lack,DepatureTime,ArrivalTime,Deliver_From,Deliver_To
1,Nissan,1,Moscow,Tambov,60,140,0,3,3,15
0,Nissan,1,Tambov,Moscow,None,None,3,6,None,None
3,Nissan,4,Moscow,Tambov,60,80,6,9,3,15
2,Nissan,4,Tambov,Moscow,None,None,9,12,None,None
5,Nissan,5,Moscow,Voronezh,60,40,12,17,17,24
4,Nissan,5,Voronezh,Moscow,None,None,17,22,None,None


В этой таблице:

- TripNumber - номер поездки, но его порядок не должен согласовываться с временем отправления. 

- Delivered - доставлено продукта в City_to.

- Lack - сколько продукта все еще требуется доставить.

- DepartureTime - время отправки из вершины.

- ArrivalTime - время прибытия в вершину.

- Deliver_From, Deliver_To - ограничения по времени доставки продукта покупателю (для справки).

#### Таблица - Доставлено покупателям, начальный спрос, неудалось доставить

In [8]:
writer.make_delivery()

,City,Delivered,Demand,Lack
0,Tambov,120,200,80
1,Voronezh,60,100,40


#### Комментарии

- Спрос превышает вместимость ТС в 5 раз. Следовательно, чтобы его удовлетворить потребутся максимум 5 поездок.

- Из-за ограничений по времени возможно сделать только 3 поездки. Переменные соответствующие поездкам 2 и 3 не используются.

- Самое выгодное сделать 2 поездки в Тамбов и одну в Воронеж.

- Спрос в обоих городах не полностью удовлетворен.

# В одном городе 2 заказа

Цель этого примера показать, как должны быть преобразованны данные, когда в одном городе 2 заказа в одном и том же окне по времени.

Здесь рассматривается одно ТС, но его вместимость чуть больше любого заказа взятого по отдельности. 

In [9]:
in_file = "tests/data/test02_v1c3split_order.xlsx"
out_file = "no_file.xlsx"

In [10]:
reader = DataReader(in_file)
processor = DataProcessor(reader)
task = LinProgMaker(processor)
task.solve()
writer = DataWriter(task, out_file)

## Данные

#### Заказы - покупатели

In [11]:
reader.read_orders()

,City,Demand,Price_per_unit,Deliver_from,Deliver_to
0,Tambov1,80,750,3,13
1,Tambov2,70,850,3,13
2,Voronezh,70,1100,17,24


#### ТС - машины

In [12]:
reader.read_vehicles()

,CarName,Capacity,Price_per_km
0,Nissan,130,9


#### Маршруты

In [13]:
reader.read_routes()

,From,To,Time,Distance
0,Moscow,Tambov1,3.000000,400
1,Moscow,Voronezh,5.000000,450
2,Tambov1,Moscow,3.000000,400
3,Tambov1,Voronezh,2.111111,190
4,Voronezh,Tambov1,2.111111,190
5,Voronezh,Moscow,5.000000,450
6,Moscow,Tambov2,3.000000,400
7,Moscow,Voronezh,5.000000,450
8,Tambov2,Moscow,3.000000,400
9,Tambov2,Voronezh,2.111111,190


## Решение и комментарии

#### Таблица - Распределение маршрутов

In [14]:
writer.make_vehicle_routing()

,CarName,TripNumber,City_From,City_To,Delivered,Lack,DepatureTime,ArrivalTime,Deliver_From,Deliver_To
2,Nissan,1,Moscow,Tambov1,60,20,0,3,3,13
1,Nissan,1,Tambov1,Tambov2,70,0,3,3.001,3,13
0,Nissan,1,Tambov2,Moscow,None,None,3.001,6.001,None,None
6,Nissan,2,Moscow,Tambov1,20,0,9.999,12.999,3,13
5,Nissan,2,Tambov1,Tambov2,0,0,12.999,13,3,13
3,Nissan,2,Tambov2,Voronezh,70,0,13,17,17,24
4,Nissan,2,Voronezh,Moscow,None,None,17,22,None,None


#### Таблица - Доставлено покупателям, начальный спрос, неудалось доставить

In [15]:
writer.make_delivery()

,City,Delivered,Demand,Lack
0,Tambov2,70,70,0
1,Tambov1,80,80,0
2,Voronezh,70,70,0


#### Комментарии

- В данном примере двойной заказ в Тамбове разделен как бы на 2 города - Тамбов1 и Тамбов2. В таблице маршрутов добавлены соответсвующие ребра. Следует обратить внимание на ребра между Тамбов1 и Тамбов2. Расстояние равно 0, и не вносит вклад в целевую функцию, а время -- небольшое число, отличное от нуля, чтобы избежать циклов.

- Из таблицы маршрутов видино, что в первом рейсе заказ в Тамбове2 выполняется полностью, а в Тамбове1 частично. Потом ТС возвращается на базу. Ждет 3 часа и едет довыполнить заказ в Тамбове1, а потом едет в Воронеж.

- Спрос полностью удовлетворен.

- Почему во втором рейсе ТС посещяет Тамбов2, а не сразу едет в Воронеж пока не знаю, но вклада в целевую функцию это точно не дает.

# 4 ТС и 6 городов

In [16]:
in_file = "tests/data/test04_v4c6.xlsx"
out_file = "no_file.xlsx"

reader = DataReader(in_file)
processor = DataProcessor(reader)
task = LinProgMaker(processor)
task.solve()
writer = DataWriter(task, out_file)

## Данные

#### Заказы - покупатели

In [17]:
reader.read_orders()

,City,Demand,Price_per_unit,Deliver_from,Deliver_to
0,Tambov,200,800,3,6
1,Voronezh,100,1100,18,22
2,Belgorod,70,1000,10,14
3,Lipetsk,250,750,12,15
4,Ryazan,120,850,9,12


#### ТС - машины

In [18]:
reader.read_vehicles()

,CarName,Capacity,Price_per_km
0,Nissan,60,9
1,Toyota,50,8
2,Mitsubishi,100,15
3,Mazda,80,10


#### Маршруты

In [19]:
reader.read_routes()

,From,To,Time,Distance
0,Voronezh,Lipetsk,1.222222,110
1,Lipetsk,Voronezh,1.222222,110
2,Tambov,Lipetsk,1.333333,120
3,Lipetsk,Tambov,1.333333,120
4,Moscow,Ryazan,2.000000,180
5,Ryazan,Moscow,2.000000,180
6,Tambov,Voronezh,2.111111,190
7,Voronezh,Tambov,2.111111,190
8,Voronezh,Belgorod,2.444444,220
9,Belgorod,Voronezh,2.444444,220


## Решение и комментарии

#### Таблица - Распределение маршрутов

In [20]:
writer.make_vehicle_routing()

,CarName,TripNumber,City_From,City_To,Delivered,Lack,DepatureTime,ArrivalTime,Deliver_From,Deliver_To
3,Mazda,1,Moscow,Tambov,80,120,0,4.44444,3,6
2,Mazda,1,Tambov,Moscow,None,None,4.44444,8.88889,None,None
4,Mazda,3,Moscow,Lipetsk,80,170,8.88889,12.8889,12,15
5,Mazda,3,Lipetsk,Moscow,None,None,12.8889,16.8889,None,None
12,Mitsubishi,1,Moscow,Tambov,100,20,0,4.44444,3,6
11,Mitsubishi,1,Tambov,Moscow,None,None,4.44444,8.88889,None,None
13,Mitsubishi,2,Moscow,Ryazan,100,20,8.88889,10.8889,9,12
14,Mitsubishi,2,Ryazan,Moscow,None,None,10.8889,12.8889,None,None
15,Mitsubishi,3,Moscow,Voronezh,100,0,12.8889,18,18,22
16,Mitsubishi,3,Voronezh,Moscow,None,None,18,23,None,None


#### Таблица - Доставлено покупателям, начальный спрос, неудалось доставить

In [21]:
writer.make_delivery()

,City,Delivered,Demand,Lack
0,Tambov,200,200,0
1,Voronezh,100,100,0
2,Belgorod,60,70,10
3,Lipetsk,130,250,120
4,Ryazan,120,120,0


#### Комментарии

- Большая часть спроса удовлетворена.

- Из-за небольшой вместимости рейсы не очень длинные.

- Это небольшая задача. Ее решение у CBC оптимизатора занимает 2 минуты. Елси добавить еще один город, то я не дождался конца решения и после 30 минут работы я остановил процесс.

- Способы ускорения работв можно найти, но выходит за рамки проекта.